# Solutions



In [ ]:
# setup SQL magic and DataJoint

import json
with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

import pymysql 
pymysql.install_as_MySQLdb()

%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

import datajoint as dj

In [ ]:
# create classes for working with tables
world = dj.create_virtual_module('world', 'shared_world')
sales = dj.create_virtual_module('sales', 'shared_sales')
sakila = dj.create_virtual_module('sakila', 'shared_sakila')
airport = dj.create_virtual_module('airport', 'shared_airport')

In [ ]:
dj.Diagram(airport)

### Homework 9 -- Chess

Chess tournament <username>_chess
Model the chess games between members of your chess clubs.

1. Club members are identified by a 12-character nickname
2. Each game has two players: one playing white and the other black.
3. Each game has a start date/time and an end date/time.
4. Each game also has one of four possible outcomes: white wins, black wins, draw, or aborted.

In [ ]:
schema = dj.Schema('dimitri_chess')

In [ ]:
@schema
class Member(dj.Manual):
    definition = """
    nickname : varchar(12)
    """
    
@schema
class Game(dj.Manual):
    definition = """
    game_id : int
    ---
    start : datetime
    end   : datetime
    -> Member.proj(white="nickname")
    -> Member.proj(black="nickname")
    """

@schema
class Outcome(dj.Manual):
    definition = """
    -> Game
    ---
    outcome : enum('white_wins',  'black_wins', 'draw', 'aborted')
    """

In [ ]:
dj.Diagram(schema)

In [ ]:
@schema
class Member2(dj.Manual):
    definition = """
    nickname : varchar(12)
    """
    
@schema
class Game2(dj.Manual):
    definition = """
    game_id : int
    ---
    start : datetime
    end   : datetime
    """

    class Player2(dj.Part):
        definition = """
        -> master
        color : enum('black', 'white')
        ---
        -> Member2
        """

@schema
class Outcome2(dj.Manual):
    definition = """
    -> Game2
    ---
    outcome : enum('white_wins',  'black_wins', 'draw', 'aborted')
    """

In [ ]:
dj.Diagram(schema)

### Payments <username>_payments
Create a database of users with their payment methods.

1. Users are identified by their usernames
2. Users have names and emails; no two users can share the same email.
3. User may have one or more payment methods.
4. A payment method is identified by a 16-bit credit card number and has a requird expiration date.
5. If a user has any payment methods, one them must be designated as the default payment methods. A user cannot have more than one default payment methods.

In [ ]:
%%sql 
create schema dimitri_payments

In [ ]:
%%sql

USE dimitri_payments

In [ ]:
%%sql

CREATE TABLE user(
    username varchar(18) NOT NULL,
    name varchar(60) NOT NULL,
    email varchar(120) NOT NULL,
    PRIMARY KEY (username)
)

In [ ]:
%%sql 
CREATE TABLE payment_method(
    credit_card_number varchar(16) NOT NULL,
    expiration_date date NOT NULL,
    is_default enum('yes'), 
    username varchar(18) NOT NULL,
    PRIMARY KEY (credit_card_number),
    FOREIGN KEY (username) REFERENCES user(username),
    UNIQUE INDEX(username, is_default)
)

In [ ]:
pay = dj.create_virtual_module('pay', 'dimitri_payments')

In [ ]:
dj.Diagram(pay)

In [ ]:
pay.schema.save('pay.py')

### Assignment 8 Problem 2

List the pairs of actors and the number of films they have been in together. Show only the top ten 10 pairs.

In [ ]:
%%sql
use shared_sakila

In [ ]:
%%sql
SELECT fa1.actor_id as actor1, fa2.actor_id as actor2, count(film_id) as n
  FROM film_actor as fa1 JOIN film_actor as fa2 USING (film_id) 
  WHERE  fa1.actor_id <  fa2.actor_id
  GROUP BY fa1.actor_id, fa2.actor_id
  ORDER BY n DESC
  LIMIT 10;

### HW 8 Problem 3
List the titles of all films on the inventory of Store 2 only, i.e. not found in Store 1.

In [ ]:
%%sql
USE shared_sakila

In [ ]:
%%sql 
SELECT * FROM inventory LIMIT 5

In [ ]:
%%sql 
SELECT DISTINCT film_id FROM inventory 
WHERE store_id = 2 and film_id NOT IN (
    SELECT film_id FROM inventory 
    WHERE store_id = 1)

### Homework 8 Problem 16
List pairs of airlines and the number of airports that they share, limit to the top 10 most connected airlines.

In [ ]:
dj.Diagram(airport)

In [ ]:
%%sql
use shared_airport

In [ ]:
%%sql

SELECT a1.airline_id, a2.airline_id, count(base_airport) as shared_airports 
  FROM airline as a1 JOIN airline as a2 USING (base_airport)
  WHERE a1.airline_id < a2.airline_id
    GROUP BY a1.airline_id, a2.airline_id

### Assignment 6 Problem 3

List all customers who have made single payments of $11.00 or more.

In [ ]:
%%sql
use shared_sakila

In [ ]:
%%sql
SELECT first_name, last_name
    FROM customer
    WHERE customer_id IN (
        SELECT customer_id FROM payment WHERE amount >= 11.0)

In [ ]:
%%sql
SELECT first_name, last_name
    FROM customer JOIN payment USING (customer_id)
    WHERE amount >= 11.0
    GROUP BY customer_id

## Homework 6  Problem 6

Display all customers who have ever ordered a bicycle

In [ ]:
%%sql
use shared_sales

In [ ]:
%%sql
SELECT DISTINCT customer_id, customer_first_name, customer_last_name
    FROM customer
        JOIN `order` USING (customer_id)
        JOIN `order__item` USING (order_number)
        JOIN product USING (product_number)
        JOIN category USING (category_id)
    WHERE (category_description = "Bikes")

In [ ]:
%%sql
SELECT customer_id, customer_first_name, customer_last_name
FROM customer WHERE customer_id IN 
    (SELECT customer_id FROM `order` 
        NATURAL JOIN `order__item` 
        NATURAL JOIN product 
        NATURAL JOIN category
    WHERE (category_description = "Bikes"))

### Homework 9 Problem 1

Create a database to keep track of each copy of a collection of book titles in your bookstore.

Book titles are identified by their ISBN.
Your store may have several copies under the same title.

In [ ]:
@schema
class BookTitle(dj.Manual):
    definition = """
    isbn : int
    ---
    title : varchar(300)
    """


@schema
class Catalog(dj.Manual):
    definition = """
    -> BookTile
    copy : tinyint unsigned 
    ---
    """